In [12]:
import json
import requests

from pathlib import Path
from getpass import getpass

from mealie.schema.recipe.recipe import Recipe

# Load Recipe

In [5]:
class RecipeWithImage(Recipe):
    image_url: str


with Path("recipe.json").open("r") as f:
    recipe = RecipeWithImage.parse_raw(f.read())

In [6]:
# recipe.dict()

# Create Recipe

In [88]:
class MealieApiClient:
    def __init__(self, base_url):
        self.base_url = base_url
        self.headers = {}
        
    @property
    def logged_in(self):
        return "access_token" in self.headers

    def to_url(self, path):
        return f"{self.base_url}{path}"

    def __getattr__(self, name):
        """
        Return proxy for requests, joining base_url with path and
        providing authentication headers automatically.
        """
        def proxy(path, **kwargs):
            url = self.to_url(path)
            set_headers = kwargs.get("headers", {})
            kwargs["headers"] = set_headers | self.headers
            return getattr(requests, name)(url, **kwargs)
        return proxy

    def fetch_api_token(self, username, password):
        login_data = {"username": username, "password": password}
        r = self.post("/auth/token", data=login_data)
        r.raise_for_status()
        return r.json()["access_token"]
        
    def login(self, username: str = "admin", password: str = ""):
        if password == "":
            password = getpass()
        access_token = self.fetch_api_token(username, password)
        self.headers={"authorization": f"Bearer {access_token}"}

    def _post_recipe_and_get_slug(self, recipe):
        r = self.post("/recipes", data=recipe.json())
        r.raise_for_status()
        slug = r.json()
        return slug

    def _scrape_image_for_recipe(self, recipe, slug):
        json_image_url = json.dumps({"url": recipe.image_url})
        scrape_image_path = f"/recipes/{slug}/image"
        r = self.post(scrape_image_path, data=json_image_url)
        r.raise_for_status()

    def _update_user_and_group_id(self, recipe, slug):
        recipe_detail_path = f"/recipes/{slug}"
        r = self.get(recipe_detail_path)
        r.raise_for_status()
        recipe_details = r.json()
        update_attributes = ["id", "userId", "groupId"]
        updated_details = {k: recipe_details[k] for k in update_attributes}
        recipe = RecipeWithImage(**(recipe.dict() | updated_details))
        return recipe

    def _update_recipe(self, recipe, slug):
        recipe_detail_path = f"/recipes/{slug}"
        r = self.put(recipe_detail_path, data=recipe.json())
        r.raise_for_status()
        return Recipe.parse_obj(r.json())
    
    def create_recipe(self, recipe):
        slug = self._post_recipe_and_get_slug(recipe)
        self._scrape_image_for_recipe(recipe, slug)
        recipe = self._update_user_and_group_id(recipe, slug)
        return self._update_recipe(recipe, slug)        

In [89]:
client = MealieApiClient("http://localhost:9000/api")

In [90]:
client.login()

 ········


In [91]:
recipe = client.create_recipe(recipe)

In [92]:
recipe.dict()

{'id': UUID('430e5bc4-de2b-41f7-b697-9c99b84ce3bf'),
 'user_id': UUID('899979e8-f240-4d53-8ed3-eaeac21ecd53'),
 'group_id': UUID('71aae446-1aa5-4db7-9782-4021692cd0ce'),
 'name': 'High Protein Herzhaftes Waffelsandwich mit Thunfisch-Creme',
 'slug': 'high-protein-herzhaftes-waffelsandwich-mit-thunfisch-creme',
 'image': None,
 'recipe_yield': None,
 'total_time': None,
 'prep_time': '30',
 'cook_time': '0',
 'perform_time': None,
 'description': '',
 'recipe_category': [],
 'tags': [],
 'tools': [],
 'rating': None,
 'org_url': None,
 'recipe_ingredient': [],
 'date_added': None,
 'date_updated': datetime.datetime(2022, 3, 26, 8, 38, 5, 686327),
 'recipe_instructions': [],
 'nutrition': {'calories': '477',
  'fat_content': '17',
  'protein_content': '53',
  'carbohydrate_content': '25',
  'fiber_content': None,
  'sodium_content': None,
  'sugar_content': None},
 'settings': {'public': False,
  'show_nutrition': False,
  'show_assets': False,
  'landscape_view': False,
  'disable_comme